In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics --upgrade

In [ ]:
# 🔧 Environment Setup & CUDA Verification
import os
import sys
import warnings
import subprocess
warnings.filterwarnings('ignore')

print("🚀 Checking system Python and CUDA installation...")

# Check PyTorch installation (should be system-wide CUDA version)
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")

    # Test CUDA availability
    cuda_available = torch.cuda.is_available()
    print(f"🔥 CUDA available: {cuda_available}")

    if cuda_available:
        print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
        print(f"🎯 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
        print(f"🎯 CUDA Version: {torch.version.cuda}")
        pytorch_working = True
    else:
        print("⚠️ CUDA not available, will use CPU")
        pytorch_working = True  # Still working, just on CPU

except ImportError as e:
    print(f"❌ PyTorch not found: {e}")
    print("Please ensure PyTorch is installed with: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
    pytorch_working = False
except Exception as e:
    print(f"❌ PyTorch error: {e}")
    pytorch_working = False

if pytorch_working:
    print("\n📦 Importing required libraries...")

    # Import other required libraries
    import yaml
    import shutil
    import random
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from pathlib import Path
    import cv2
    from PIL import Image
    import pandas as pd
    from sklearn.model_selection import train_test_split
    import glob
    from collections import Counter
    import json

    # Set up device and optimization
    if torch.cuda.is_available():
        # GPU optimization
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
        torch.cuda.empty_cache()

        DEVICE = 'cuda'
        USE_GPU = True
        print(f"🚀 Using GPU acceleration on {torch.cuda.get_device_name(0)}!")
    else:
        DEVICE = 'cpu'
        USE_GPU = False
        print("⚠️ Using CPU (CUDA not available)")

    # Import YOLOv8 (assuming it's already installed as per the previous cell)
    try:
        from ultralytics import YOLO
        print("✅ YOLOv8 imported successfully")
    except ImportError:
         print("❌ YOLOv8 not found. Please ensure it's installed with: pip install ultralytics")


    # Set random seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    if USE_GPU:
        torch.cuda.manual_seed_all(42)

    print(f"\n🚀 Environment ready!")
    print(f"   Device: {DEVICE}")
    print(f"   Working Directory: {os.getcwd()}")
    print(f"   Python: {sys.executable}")

    # List available folders
    available_folders = [d.name for d in Path('.').iterdir() if d.is_dir()]
    print(f"   Available folders: {available_folders}")

else:
    print("❌ Environment setup failed. Please ensure PyTorch with CUDA is installed.")

In [ ]:
# 📂 Dataset Configuration & Path Setup
# Model Configuration - Using YOLOv8x for maximum accuracy
SELECTED_MODEL = 'yolov8x'  # Best accuracy accuracy
EPOCHS = 300  # More epochs for better accuracy
BATCH_SIZE = 8 if USE_GPU else 4  # Adjusted for YOLOv8x memory requirements
IMG_SIZE = 1024  # Larger image size for better QR detection
PATIENCE = 50
CONFIDENCE_THRESHOLD = 0.1  # Lower threshold for better recall
IOU_THRESHOLD = 0.5
LEARNING_RATE = 0.0005  # Lower LR for better convergence

# Dataset Paths - Exact structure you mentioned
DATASET_ROOT = Path("/content/drive/MyDrive/QR_Dataset/QR_Dataset")
TRAIN_IMAGES = DATASET_ROOT / "/Users/Kareem/Documents/1Pharmacy/QR_Dataset/train_images"
TRAIN_LABELS = Path("/Users/Kareem/Documents/1Pharmacy/QR_Dataset/train_label")  # Updated path
TEST_IMAGES = DATASET_ROOT / "/Users/Kareem/Documents/1Pharmacy/QR_Dataset/test_images"
TEST_LABELS = Path("/Users/Kareem/Documents/1Pharmacy/QR_Dataset/test_label")    # Updated path

print(f"🎯 Configuration:")
print(f"   Model: {SELECTED_MODEL} (Maximum Accuracy)")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Image Size: {IMG_SIZE}")
print(f"   Device: {DEVICE}")
print(f"   Learning Rate: {LEARNING_RATE}")

print(f"\n📂 Dataset Structure:")
print(f"   Train Images: {TRAIN_IMAGES}")
print(f"   Train Labels: {TRAIN_LABELS}")
print(f"   Test Images: {TEST_IMAGES}")
print(f"   Test Labels: {TEST_LABELS}")

# Verify dataset exists
for path in [TRAIN_IMAGES, TRAIN_LABELS, TEST_IMAGES, TEST_LABELS]:
    if path.exists():
        print(f"   ✅ {path.name}: Found")
    else:
        print(f"   ❌ {path.name}: Not found")
        # Removed mkdir since the user expects the data to be in the drive
        # path.mkdir(parents=True, exist_ok=True)
        # print(f"   📁 Created {path}")


# Count files
train_imgs = list(TRAIN_IMAGES.glob('*.jpg')) + list(TRAIN_IMAGES.glob('*.png'))
train_labels = [f for f in TRAIN_LABELS.glob('*.txt') if f.name != 'classes.txt']
test_imgs = list(TEST_IMAGES.glob('*.jpg')) + list(TEST_IMAGES.glob('*.png'))
test_labels = [f for f in TEST_LABELS.glob('*.txt') if f.name != 'classes.txt']


print(f"\n📊 Dataset Summary:")
print(f"   Training Images: {len(train_imgs)}")
print(f"   Training Labels: {len(train_labels)}")
print(f"   Test Images: {len(test_imgs)}")
print(f"   Test Labels: {len(test_labels)}")

if len(train_imgs) == 0:
    print("⚠️ No training images found! Please add images to train_images folder.")
if len(train_labels) == 0:
    print("⚠️ No training labels found! Please add LabelImg annotations to train_label folder.")

In [ ]:
# 📂 Dataset Configuration & Path Setup
# Model Configuration - Using YOLOv8x for maximum accuracy
SELECTED_MODEL = 'yolov8x'  # Best accuracy accuracy
EPOCHS = 300  # More epochs for better accuracy
BATCH_SIZE = 8 if USE_GPU else 4  # Adjusted for YOLOv8x memory requirements
IMG_SIZE = 1024  # Larger image size for better QR detection
PATIENCE = 50
CONFIDENCE_THRESHOLD = 0.1  # Lower threshold for better recall
IOU_THRESHOLD = 0.5
LEARNING_RATE = 0.0005  # Lower LR for better convergence

# Dataset Paths - Adjusted for Google Drive
# Assumes your dataset is in /content/drive/MyDrive/QR_Dataset/QR_Dataset
DATASET_ROOT = Path("/Users/Kareem/Documents/1Pharmacy/QR_Dataset")
TRAIN_IMAGES = DATASET_ROOT / "train_images"
TRAIN_LABELS = DATASET_ROOT / "train_label"  # Assuming labels are in an annotation folder at dataset root
TEST_IMAGES = DATASET_ROOT / "test_images"
TEST_LABELS = DATASET_ROOT / "test_label"    # Assuming labels are in an annotation folder at dataset root

print(f"🎯 Configuration:")
print(f"   Model: {SELECTED_MODEL} (Maximum Accuracy)")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Image Size: {IMG_SIZE}")
print(f"   Device: {DEVICE}")
print(f"   Learning Rate: {LEARNING_RATE}")

print(f"\n📂 Dataset Structure:")
print(f"   Train Images: {TRAIN_IMAGES}")
print(f"   Train Labels: {TRAIN_LABELS}")
print(f"   Test Images: {TEST_IMAGES}")
print(f"   Test Labels: {TEST_LABELS}")

# Verify dataset exists
for path in [TRAIN_IMAGES, TRAIN_LABELS, TEST_IMAGES, TEST_LABELS]:
    if path.exists():
        print(f"   ✅ {path.name}: Found")
    else:
        print(f"   ❌ {path.name}: Not found")
        # Removed mkdir since the user expects the data to be in the drive
        # path.mkdir(parents=True, exist_ok=True)
        # print(f"   📁 Created {path}")


# Count files
train_imgs = list(TRAIN_IMAGES.glob('*.jpg')) + list(TRAIN_IMAGES.glob('*.png')) + list(TRAIN_IMAGES.glob('*.jpeg'))
train_labels = [f for f in TRAIN_LABELS.glob('*.txt') if f.name != 'classes.txt']
test_imgs = list(TEST_IMAGES.glob('*.jpg')) + list(TEST_IMAGES.glob('*.png')) + list(TEST_IMAGES.glob('*.jpeg'))
test_labels = [f for f in TEST_LABELS.glob('*.txt') if f.name != 'classes.txt']


print(f"\n📊 Dataset Summary:")
print(f"   Training Images: {len(train_imgs)}")
print(f"   Training Labels: {len(train_labels)}")
print(f"   Test Images: {len(test_imgs)}")
print(f"   Test Labels: {len(test_labels)}")

if len(train_imgs) == 0:
    print("⚠️ No training images found! Please add images to train_images folder.")
if len(train_labels) == 0:
    print("⚠️ No training labels found! Please add LabelImg annotations to train_label folder.")

In [ ]:
# 🔧 Label Processing & Class Normalization
def process_and_normalize_labels():
    """
    Process all label files and convert all classes to single QR class (0)
    Handles multiple QR codes per image and fixes annotation issues
    """
    print("🔧 Processing and normalizing labels...")

    # Do not create classes.txt here. This should be in the dataset root
    # or defined in the data.yaml file for training.
    # classes_file = TRAIN_LABELS / "classes.txt"
    # with open(classes_file, 'w') as f:
    #     f.write('QR_Code\n')

    # Also create for test labels
    # test_classes_file = TEST_LABELS / "classes.txt"
    # with open(test_classes_file, 'w') as f:
    #     f.write('QR_Code\n')

    print("ℹ️ Class definition will be handled in the data.yaml file.")

    # Process training labels
    processed_count = 0
    fixed_count = 0
    total_objects = 0

    for label_file in TRAIN_LABELS.glob("*.txt"):
        if label_file.name == "classes.txt":
            continue

        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()

            # Check if the file is empty
            if not lines:
                print(f"⚠️ Skipping empty label file: {label_file.name}")
                continue

            processed_lines = []
            for line in lines:
                line = line.strip()
                if not line:
                    continue

                parts = line.split()
                if len(parts) >= 5:
                    # Convert any class ID to 0 (QR_Code)
                    original_class = int(parts[0])
                    if original_class != 0:
                        fixed_count += 1

                    # Normalize coordinates (ensure they're within 0-1 range)
                    x_center = max(0.0, min(1.0, float(parts[1])))
                    y_center = max(0.0, min(1.0, float(parts[2])))
                    width = max(0.001, min(1.0, float(parts[3])))  # Minimum width to avoid zero-size boxes
                    height = max(0.001, min(1.0, float(parts[4])))  # Minimum height to avoid zero-size boxes

                    # Convert to class 0 (QR_Code)
                    processed_line = f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    processed_lines.append(processed_line)
                    total_objects += 1
                else:
                    print(f"⚠️ Skipping malformed line in {label_file.name}: {line}")


            # Write back processed labels
            if processed_lines:
                with open(label_file, 'w') as f:
                    f.write('\n'.join(processed_lines) + '\n')
                processed_count += 1
            # If processed_lines is empty after filtering, the file might have contained only malformed lines
            elif lines:
                 print(f"⚠️ No valid labels found after processing {label_file.name}. File might contain only malformed lines.")


        except Exception as e:
            print(f"⚠️ Error processing {label_file.name}: {e}")

    # Process test labels similarly
    test_processed = 0
    for label_file in TEST_LABELS.glob("*.txt"):
        if label_file.name == "classes.txt":
            continue

        try:
            with open(label_file, 'r') as f:
                lines = f.readlines()

            # Check if the file is empty
            if not lines:
                print(f"⚠️ Skipping empty test label file: {label_file.name}")
                continue


            processed_lines = []
            for line in lines:
                line = line.strip()
                if not line:
                    continue

                parts = line.split()
                if len(parts) >= 5:
                    # Convert to class 0 and normalize
                    x_center = max(0.0, min(1.0, float(parts[1])))
                    y_center = max(0.0, min(1.0, float(parts[2])))
                    width = max(0.001, min(1.0, float(parts[3])))
                    height = max(0.001, min(1.0, float(parts[4])))

                    processed_line = f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    processed_lines.append(processed_line)
                else:
                    print(f"⚠️ Skipping malformed line in test {label_file.name}: {line}")


            if processed_lines:
                with open(label_file, 'w') as f:
                    f.write('\n'.join(processed_lines) + '\n')
                test_processed += 1
            # If processed_lines is empty after filtering, the file might have contained only malformed lines
            elif lines:
                 print(f"⚠️ No valid test labels found after processing {label_file.name}. File might contain only malformed lines.")


        except Exception as e:
            print(f"⚠️ Error processing test {label_file.name}: {e}")

    print(f"📊 Label Processing Results:")
    print(f"   Training labels processed: {processed_count}")
    print(f"   Test labels processed: {test_processed}")
    print(f"   Total QR objects: {total_objects}")
    print(f"   Class IDs fixed: {fixed_count}")
    print(f"✅ All labels normalized to single QR class (0)")

    return total_objects

# Run label processing
total_qr_objects = process_and_normalize_labels()

In [ ]:
# 📊 Dataset Analysis & Validation
def analyze_dataset():
    """Comprehensive dataset analysis"""
    print("🔍 Analyzing dataset...")

    # Analyze training data
    train_images = list(TRAIN_IMAGES.glob('*.jpg')) + list(TRAIN_IMAGES.glob('*.png'))
    train_labels = [f for f in TRAIN_LABELS.glob('*.txt') if f.name != 'classes.txt']

    # Check image-label pairs
    matched_pairs = 0
    missing_labels = []
    missing_images = []

    for img_file in train_images:
        label_file = TRAIN_LABELS / f"{img_file.stem}.txt"
        if label_file.exists():
            matched_pairs += 1
        else:
            missing_labels.append(img_file.name)

    for label_file in train_labels:
        img_jpg = TRAIN_IMAGES / f"{label_file.stem}.jpg"
        img_png = TRAIN_IMAGES / f"{label_file.stem}.png"
        if not (img_jpg.exists() or img_png.exists()):
            missing_images.append(label_file.name)

    # Analyze QR code statistics
    qr_counts = []
    bbox_areas = []

    for label_file in train_labels:
        try:
            with open(label_file, 'r') as f:
                lines = [line.strip() for line in f.readlines() if line.strip()]

            qr_count = len(lines)
            qr_counts.append(qr_count)

            for line in lines:
                parts = line.split()
                if len(parts) >= 5:
                    width = float(parts[3])
                    height = float(parts[4])
                    area = width * height
                    bbox_areas.append(area)
        except:
            continue

    print(f"📊 Dataset Analysis Results:")
    print(f"   Total training images: {len(train_images)}")
    print(f"   Total training labels: {len(train_labels)}")
    print(f"   Matched image-label pairs: {matched_pairs}")
    print(f"   Missing labels: {len(missing_labels)}")
    print(f"   Missing images: {len(missing_images)}")

    if qr_counts:
        print(f"\n🎯 QR Code Statistics:")
        print(f"   Total QR codes: {sum(qr_counts)}")
        print(f"   Images with QR codes: {len(qr_counts)}")
        print(f"   Avg QR codes per image: {np.mean(qr_counts):.2f}")
        print(f"   Max QR codes in single image: {max(qr_counts)}")
        print(f"   Min QR codes in single image: {min(qr_counts)}")

    if bbox_areas:
        print(f"\n📐 Bounding Box Analysis:")
        print(f"   Avg area: {np.mean(bbox_areas):.4f}")
        print(f"   Min area: {np.min(bbox_areas):.4f}")
        print(f"   Max area: {np.max(bbox_areas):.4f}")
        print(f"   Median area: {np.median(bbox_areas):.4f}")

    # Print warnings
    if missing_labels:
        print(f"⚠️ Images without labels: {missing_labels[:5]}{'...' if len(missing_labels) > 5 else ''}")
    if missing_images:
        print(f"⚠️ Labels without images: {missing_images[:5]}{'...' if len(missing_images) > 5 else ''}")

    return {
        'total_images': len(train_images),
        'total_labels': len(train_labels),
        'matched_pairs': matched_pairs,
        'total_qr_codes': sum(qr_counts) if qr_counts else 0,
        'avg_qr_per_image': np.mean(qr_counts) if qr_counts else 0
    }

# Run dataset analysis
dataset_stats = analyze_dataset()

In [ ]:
# 📁 YOLO Dataset Preparation
def prepare_yolo_dataset():
    """Prepare dataset in YOLO format with train/val split"""
    print("📁 Preparing YOLO dataset structure...")

    # Create dataset directory structure
    dataset_dir = Path("./yolo_qr_dataset")

    # Remove existing dataset to start fresh
    if dataset_dir.exists():
        shutil.rmtree(dataset_dir)

    # Create directory structure
    for split in ['train', 'val']:
        for folder in ['images', 'labels']:
            (dataset_dir / split / folder).mkdir(parents=True, exist_ok=True)

    # Get all image files with corresponding labels
    train_images = []
    for img_ext in ['*.jpg', '*.png', '*.jpeg']:
        train_images.extend(list(TRAIN_IMAGES.glob(img_ext)))

    # Filter images that have corresponding labels
    valid_images = []
    for img_file in train_images:
        label_file = TRAIN_LABELS / f"{img_file.stem}.txt"
        if label_file.exists():
            valid_images.append(img_file)

    print(f"   Total images: {len(train_images)}")
    print(f"   Images with labels: {len(valid_images)}")

    if len(valid_images) == 0:
        raise ValueError("No valid image-label pairs found!")

    # Split dataset (80% train, 20% validation)
    train_files, val_files = train_test_split(
        valid_images,
        train_size=0.8,
        random_state=42,
        shuffle=True
    )

    print(f"   Training split: {len(train_files)} images")
    print(f"   Validation split: {len(val_files)} images")

    # Copy files to respective directories
    def copy_files(file_list, split_name):
        for img_file in file_list:
            # Copy image
            dst_img = dataset_dir / split_name / "images" / img_file.name
            shutil.copy2(img_file, dst_img)

            # Copy corresponding label
            label_file = TRAIN_LABELS / f"{img_file.stem}.txt"
            dst_label = dataset_dir / split_name / "labels" / f"{img_file.stem}.txt"
            shutil.copy2(label_file, dst_label)

    copy_files(train_files, "train")
    copy_files(val_files, "val")

    # Create data.yaml configuration file
    data_yaml = {
        'path': str(dataset_dir.absolute()),
        'train': 'train/images',
        'val': 'val/images',
        'nc': 1,  # Number of classes (single QR class)
        'names': {0: 'QR_Code'}  # Class names
    }

    yaml_path = dataset_dir / "data.yaml"
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

    print(f"✅ YOLO dataset prepared!")
    print(f"   Dataset path: {dataset_dir}")
    print(f"   Configuration: {yaml_path}")

    return dataset_dir

# Prepare the dataset
dataset_dir = prepare_yolo_dataset()

In [ ]:
# 🎯 Maximum Accuracy YOLOv8x Training
def train_maximum_accuracy_model():
    """Train YOLOv8x with hyperparameters optimized for maximum QR detection accuracy"""

    # Load the most powerful YOLO model - Using the local path provided by the user
    # Reduced model size to yolov8s for better performance on 4GB GPU
    SELECTED_MODEL = 'yolov8s'
    print(f"🚀 Starting {SELECTED_MODEL} training for maximum QR detection accuracy...")

    # Use the user-provided local model path
    model_path = Path("/Users/Kareem/Documents/1Pharmacy/yolov8s.pt")

    if not model_path.exists():
        print(f"❌ Model weights not found at the specified path: {model_path}")
        print("Please ensure the file exists at this location.")
        return None, None # Exit if the model is not found

    print(f"Attempting to load model from: {model_path}")
    model = YOLO(str(model_path)) # Convert Path object to string
    print(f"✅ Loaded {SELECTED_MODEL} pre-trained model")


    # Clear GPU memory
    if USE_GPU:
        torch.cuda.empty_cache()
        print(f"🧹 Cleared GPU memory")

    # Training arguments optimized for maximum accuracy
    training_args = {
        # Dataset configuration
        'data': str(dataset_dir / 'data.yaml'), # Ensure this path is correct for local runtime if data is also local
        'epochs': EPOCHS,  # Increased epochs back to original for accuracy
        'batch': 1 if USE_GPU else 4, # Further reduced batch size to mitigate CUDA out of memory
        'imgsz': 640,  # Reduced image size
        'device': DEVICE,
        'workers': 0, # Reduced workers to 0 to avoid multiprocessing issues on Windows

        # Model settings
        # 'model': SELECTED_MODEL, # No longer needed as we load from path
        'patience': PATIENCE,
        'save_period': 20,  # Save checkpoint every 20 epochs
        'project': 'qr_detection_max_accuracy',
        'name': f'qr_{SELECTED_MODEL}_best_accuracy',
        'exist_ok': True,
        'verbose': True,
        'seed': 42,
        'deterministic': True,
        'single_cls': True,  # Single class optimization

        # Optimizer settings for maximum accuracy
        'optimizer': 'AdamW',  # Better convergence than SGD
        'lr0': LEARNING_RATE,  # Lower learning rate for stability
        'lrf': 0.001,  # Very low final learning rate
        'momentum': 0.9,
        'weight_decay': 0.001,  # Stronger regularization
        'warmup_epochs': 5.0,  # More warmup epochs
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,

        # Data augmentation optimized for QR codes (minimal to preserve structure)
        'hsv_h': 0.005,    # Very minimal hue changes
        'hsv_s': 0.2,      # Moderate saturation changes
        'hsv_v': 0.2,      # Moderate brightness changes
        'degrees': 5.0,    # Very small rotations only
        'translate': 0.05, # Minimal translation
        'scale': 0.1,      # Minimal scaling
        'shear': 1.0,      # Minimal shear
        'perspective': 0.0, # No perspective transform
        'flipud': 0.0,     # No vertical flipping
        'fliplr': 0.0,     # No horizontal flipping
        'mosaic': 0.3,     # Reduced mosaic for better QR preservation
        'mixup': 0.0,      # No mixup to preserve QR structure
        'copy_paste': 0.0, # No copy-paste
        'auto_augment': None, # Disable auto augmentation
        'erasing': 0.0,    # No random erasing

        # Loss function weights (optimized for detection accuracy)
        'box': 10.0,       # Higher weight for box regression
        'cls': 1.0,        # Standard classification weight
        'dfl': 2.0,        # Higher DFL weight for precise localization
        'pose': 12.0,      # Pose weight (if applicable)
        'kobj': 1.0,       # Keypoint objectness
        'label_smoothing': 0.0,  # No label smoothing for precise QR detection

        # Validation and evaluation
        'val': True,
        'split': 'val',
        'save_json': True,
        'save_hybrid': True,
        'conf': CONFIDENCE_THRESHOLD,
        'iou': IOU_THRESHOLD,
        'max_det': 100,    # Allow more detections per image for multiple QRs
        'half': USE_GPU,     # Use FP16 for mixed precision if GPU is available
        'dnn': False,
        'plots': True,
        'save': True,
        'save_txt': True,
        'save_conf': True,
        'save_crop': False,
        'show_labels': True,
        'show_conf': True,
        'show_boxes': True,

        # Advanced settings for accuracy
        'amp': False,      # Disable mixed precision to prevent unwanted downloads
        'fraction': 1.0,   # Use 100% of dataset
        'profile': False,
        'freeze': None,    # Don't freeze any layers
        'multi_scale': True,  # Enable multi-scale training
        'overlap_mask': True,
        'mask_ratio': 4,
        'dropout': 0.1,    # Slight dropout for regularization
        # 'val_period': 5,   # Validate every 5 epochs - REMOVED, NOT A VALID ARGUMENT
    }

    print(f"🎯 Training Configuration:")
    print(f"   Model: {SELECTED_MODEL}")
    print(f"   Epochs: {EPOCHS}")
    print(f"   Batch Size: {training_args['batch']}") # Print the actual batch size being used
    print(f"   Image Size: {IMG_SIZE}")
    print(f"   Learning Rate: {LEARNING_RATE}")
    print(f"   Device: {DEVICE}")
    print(f"   Mixed Precision: {'Enabled' if USE_GPU else 'Disabled'} (based on GPU availability)")

    try:
        # Start training
        print("🔥 Starting training...")
        results = model.train(**training_args)

        print("✅ Training completed successfully!")

        # Get training results
        metrics = results.results_dict if hasattr(results, 'results_dict') else {}

        print(f"\n📊 Training Results:")
        if metrics:
            for key, value in metrics.items():
                if isinstance(value, (int, float)):
                    print(f"   {key}: {value:.4f}")

        return model, results

    except Exception as e:
        print(f"❌ Training failed: {e}")
        return None, None

# Start training
print("🚀 Initiating maximum accuracy QR detection training...")
trained_model, training_results = train_maximum_accuracy_model()

if trained_model is not None:
    print("\n🎉 Training pipeline completed successfully!")
    print("📁 Check 'qr_detection_max_accuracy' folder for results")
else:
    print("\n❌ Training failed. Please check the error messages above.")

In [ ]:
# 🖼️ Visualization & Testing
def visualize_predictions(model_path, num_samples=6):
    """Visualize model predictions on sample images"""
    try:
        model = YOLO(model_path)
        print(f"✅ Loaded model from {model_path}")
    except Exception as e:
        print(f"❌ Failed to load model from {model_path}: {e}")
        return

    print("🖼️ Visualizing QR detection results...")

    # Get validation images
    val_images = list((dataset_dir / "val" / "images").glob("*"))
    if not val_images:
        print("❌ No validation images found")
        return

    # Select random samples
    sample_images = random.sample(val_images, min(num_samples, len(val_images)))

    # Create visualization
    cols = 3
    rows = (len(sample_images) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()

    for idx, img_path in enumerate(sample_images):
        # Load and process image
        image = cv2.imread(str(img_path))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Run prediction
        results = model.predict(
            str(img_path),
            conf=CONFIDENCE_THRESHOLD,
            iou=IOU_THRESHOLD,
            imgsz=IMG_SIZE,
            device=DEVICE,
            verbose=False,
            save=False
        )

        # Draw predictions
        if len(results) > 0 and results[0].boxes is not None:
            boxes = results[0].boxes
            for i in range(len(boxes)):
                # Get box coordinates
                x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy().astype(int)
                conf = boxes.conf[i].cpu().numpy()

                # Draw bounding box
                cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (0, 255, 0), 3)

                # Add confidence label
                label = f'QR: {conf:.2f}'
                cv2.putText(image_rgb, label, (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display image
        if idx < len(axes):
            axes[idx].imshow(image_rgb)
            axes[idx].set_title(f"Sample {idx+1}: {img_path.name}")
            axes[idx].axis('off')

    # Hide unused subplots
    for idx in range(len(sample_images), len(axes)):
        axes[idx].set_visible(False)

    plt.tight_layout()
    plt.suptitle('🎯 QR Detection Results', fontsize=16, fontweight='bold', y=0.98)
    plt.show()

def test_on_test_images(model_path):
    """Test model on test images"""
    try:
        model = YOLO(model_path)
        print(f"✅ Loaded model from {model_path}")
    except Exception as e:
        print(f"❌ Failed to load model from {model_path}: {e}")
        return

    print("🧪 Testing on test images...")

    test_images = list(TEST_IMAGES.glob('*.jpg')) + list(TEST_IMAGES.glob('*.png'))
    if not test_images:
        print("❌ No test images found")
        return

    total_detections = 0
    images_with_qr = 0

    print(f"Testing on {len(test_images)} images...")

    for img_path in test_images[:10]:  # Test on first 10 images
        results = model.predict(
            str(img_path),
            conf=CONFIDENCE_THRESHOLD,
            iou=IOU_THRESHOLD,
            imgsz=IMG_SIZE,
            device=DEVICE,
            verbose=False,
            save=False
        )

        if len(results) > 0 and results[0].boxes is not None:
            num_qr = len(results[0].boxes)
            if num_qr > 0:
                images_with_qr += 1
                total_detections += num_qr
                print(f"   {img_path.name}: {num_qr} QR codes detected")
        else:
            print(f"   {img_path.name}: No QR codes detected")

    print(f"\n📊 Test Results:")
    print(f"   Images with QR detected: {images_with_qr}/{min(10, len(test_images))}")
    print(f"   Total QR codes detected: {total_detections}")
    print(f"   Average QR per image: {total_detections/min(10, len(test_images)):.2f}")

# Run visualizations and testing
# Use the path to the best weights from the training run
best_model_path = "/qr_detection_max_accuracy/qr_yolov8s_best_accuracy/weights/best.pt"

# Check if the trained_model variable exists and has the best weights path
if 'trained_model' in globals() and trained_model is not None:
    # Use the path from the training results if available
    try:
        best_model_path = str(Path(trained_model.save_dir) / "weights" / "best.pt")
        print(f"Using best model path from training results: {best_model_path}")
    except Exception as e:
        print(f"Could not get best model path from training results, using default: {best_model_path}")
        print(f"Error: {e}")


visualize_predictions(best_model_path)
test_on_test_images(best_model_path)

In [ ]:
# 📈 Training Curves & Performance Analysis
def plot_training_curves(experiment_path):
    """Plot detailed training curves and metrics"""
    if experiment_path is None:
        print("❌ No experiment path available")
        return

    print("📈 Plotting training curves...")

    results_file = experiment_path / "results.csv"
    if not results_file.exists():
        print(f"❌ Results CSV not found at {results_file}")
        return

    # Load training results
    df = pd.read_csv(results_file)
    df.columns = df.columns.str.strip()

    # Create comprehensive plots
    fig, axes = plt.subplots(3, 2, figsize=(15, 18))
    fig.suptitle('🎯 YOLOv8 QR Detection Training Analysis', fontsize=16, fontweight='bold')

    # Loss curves
    axes[0, 0].plot(df.index, df['train/box_loss'], 'b-', label='Train Box Loss', linewidth=2)
    axes[0, 0].plot(df.index, df['val/box_loss'], 'r-', label='Val Box Loss', linewidth=2)
    axes[0, 0].set_title('Box Regression Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].plot(df.index, df['train/cls_loss'], 'b-', label='Train Cls Loss', linewidth=2)
    axes[0, 1].plot(df.index, df['val/cls_loss'], 'r-', label='Val Cls Loss', linewidth=2)
    axes[0, 1].set_title('Classification Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # DFL Loss
    if 'train/dfl_loss' in df.columns:
        axes[1, 0].plot(df.index, df['train/dfl_loss'], 'b-', label='Train DFL Loss', linewidth=2)
        axes[1, 0].plot(df.index, df['val/dfl_loss'], 'r-', label='Val DFL Loss', linewidth=2)
        axes[1, 0].set_title('Distribution Focal Loss (DFL)')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Loss')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

    # Precision and Recall
    if 'metrics/precision(B)' in df.columns:
        axes[1, 1].plot(df.index, df['metrics/precision(B)'], 'g-', label='Precision', linewidth=2)
        axes[1, 1].plot(df.index, df['metrics/recall(B)'], 'orange', label='Recall', linewidth=2)
        axes[1, 1].set_title('Precision & Recall')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Value')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].set_ylim([0, 1])

    # mAP metrics
    if 'metrics/mAP50(B)' in df.columns:
        axes[2, 0].plot(df.index, df['metrics/mAP50(B)'], 'purple', label='mAP@0.5', linewidth=2)
        if 'metrics/mAP50-95(B)' in df.columns:
            axes[2, 0].plot(df.index, df['metrics/mAP50-95(B)'], 'brown', label='mAP@0.5:0.95', linewidth=2)
        axes[2, 0].set_title('Mean Average Precision (mAP)')
        axes[2, 0].set_xlabel('Epoch')
        axes[2, 0].set_ylabel('mAP')
        axes[2, 0].legend()
        axes[2, 0].grid(True, alpha=0.3)
        axes[2, 0].set_ylim([0, 1])

    # Learning rate
    if 'lr/pg0' in df.columns:
        axes[2, 1].plot(df.index, df['lr/pg0'], 'cyan', label='Learning Rate', linewidth=2)
        axes[2, 1].set_title('Learning Rate Schedule')
        axes[2, 1].set_xlabel('Epoch')
        axes[2, 1].set_ylabel('Learning Rate')
        axes[2, 1].legend()
        axes[2, 1].grid(True, alpha=0.3)
        axes[2, 1].set_yscale('log')

    plt.tight_layout()
    plt.show()

    # Print final metrics summary
    if len(df) > 0:
        final_metrics = df.iloc[-1]
        print(f"\n📊 Final Training Metrics:")

        metrics_to_show = [
            ('metrics/precision(B)', 'Precision'),
            ('metrics/recall(B)', 'Recall'),
            ('metrics/mAP50(B)', 'mAP@0.5'),
            ('metrics/mAP50-95(B)', 'mAP@0.5:0.95'),
            ('val/box_loss', 'Validation Box Loss'),
            ('val/cls_loss', 'Validation Cls Loss')
        ]

        for col, name in metrics_to_show:
            if col in df.columns:
                value = final_metrics[col]
                print(f"   {name}: {value:.4f}")

        # Best mAP50 epoch
        if 'metrics/mAP50(B)' in df.columns:
            best_map50_epoch = df['metrics/mAP50(B)'].idxmax()
            best_map50_value = df['metrics/mAP50(B)'].max()
            print(f"\n🏆 Best Performance:")
            print(f"   Best mAP@0.5: {best_map50_value:.4f} (Epoch {best_map50_epoch + 1})")

# Plot training analysis
if 'training_results' in globals() and training_results is not None:
    experiment_path = Path(training_results.save_dir)
    plot_training_curves(experiment_path)
else:
    print("❌ Training results not available to plot curves.")

In [ ]:
# 🚀 Model Export & Production Deployment
def export_production_models(model):
    """Export trained model in multiple formats for production deployment"""
    if model is None:
        print("❌ No model available for export")
        return

    print("🚀 Exporting model for production deployment...")

    export_results = {}
    export_formats = ['onnx', 'torchscript', 'openvino']

    for format_type in export_formats:
        try:
            print(f"📦 Exporting to {format_type.upper()}...")

            if format_type == 'onnx':
                # ONNX export for cross-platform deployment
                export_path = model.export(
                    format='onnx',
                    imgsz=IMG_SIZE,
                    half=False,  # FP32 for maximum compatibility
                    int8=False,
                    dynamic=True,  # Dynamic input shapes
                    simplify=True,
                    opset=17,  # Latest ONNX opset
                    verbose=True
                )

            elif format_type == 'torchscript':
                # TorchScript for PyTorch deployment
                export_path = model.export(
                    format='torchscript',
                    imgsz=IMG_SIZE,
                    optimize=True,
                    half=False,
                    verbose=True
                )

            elif format_type == 'openvino':
                # OpenVINO for Intel hardware optimization
                export_path = model.export(
                    format='openvino',
                    imgsz=IMG_SIZE,
                    half=False,
                    int8=False,
                    verbose=True
                )

            export_results[format_type] = export_path
            print(f"   ✅ {format_type.upper()} export successful: {export_path}")

        except Exception as e:
            print(f"   ❌ {format_type.upper()} export failed: {str(e)}")
            export_results[format_type] = None

    print(f"\n📦 Export Summary:")
    for format_type, path in export_results.items():
        if path:
            print(f"   ✅ {format_type.upper()}: {path}")
        else:
            print(f"   ❌ {format_type.upper()}: Export failed")

    return export_results

def create_inference_function(model):
    """Create a production-ready inference function"""
    if model is None:
        print("❌ No model available")
        return None

    def detect_qr_codes(image_path, conf_threshold=0.25, iou_threshold=0.5):
        """
        Detect QR codes in an image

        Args:
            image_path: Path to the image file
            conf_threshold: Confidence threshold for detection
            iou_threshold: IoU threshold for NMS

        Returns:
            List of detections with bounding boxes and confidence scores
        """
        try:
            # Run prediction
            results = model.predict(
                source=image_path,
                conf=conf_threshold,
                iou=iou_threshold,
                imgsz=IMG_SIZE,
                device=DEVICE,
                verbose=False,
                save=False
            )

            detections = []
            if len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes

                for i in range(len(boxes)):
                    # Get detection data
                    bbox = boxes.xyxy[i].cpu().numpy()  # [x1, y1, x2, y2]
                    conf = boxes.conf[i].cpu().numpy()

                    detection = {
                        'bbox': bbox.tolist(),
                        'confidence': float(conf),
                        'class': 'QR_Code',
                        'x1': float(bbox[0]),
                        'y1': float(bbox[1]),
                        'x2': float(bbox[2]),
                        'y2': float(bbox[3]),
                        'width': float(bbox[2] - bbox[0]),
                        'height': float(bbox[3] - bbox[1])
                    }
                    detections.append(detection)

            return detections

        except Exception as e:
            print(f"Error during inference: {e}")
            return []

    return detect_qr_codes

def save_model_info(experiment_path):
    """Save model configuration and performance info"""
    if experiment_path is None:
        return

    model_info = {
        'model_type': SELECTED_MODEL,
        'training_config': {
            'epochs': EPOCHS,
            'batch_size': BATCH_SIZE,
            'image_size': IMG_SIZE,
            'learning_rate': LEARNING_RATE,
            'device': DEVICE
        },
        'dataset_info': dataset_stats,
        'paths': {
            'train_images': str(TRAIN_IMAGES),
            'train_labels': str(TRAIN_LABELS),
            'test_images': str(TEST_IMAGES),
            'test_labels': str(TEST_LABELS)
        },
        'usage_instructions': {
            'inference': 'Use detect_qr_codes(image_path) function',
            'confidence_threshold': CONFIDENCE_THRESHOLD,
            'iou_threshold': IOU_THRESHOLD,
            'supported_formats': ['jpg', 'png', 'jpeg'],
            'output_format': 'List of detections with bbox coordinates and confidence'
        }
    }

    info_file = experiment_path / 'model_info.json'
    with open(info_file, 'w') as f:
        json.dump(model_info, f, indent=2)

    print(f"📄 Model info saved: {info_file}")


# Export models and create inference function
if 'trained_model' in globals() and trained_model is not None:
    # Export to different formats
    export_results = export_production_models(trained_model)

    # Create inference function
    qr_detector = create_inference_function(trained_model)

    # Save model information
    if 'training_results' in globals() and training_results is not None:
        experiment_path = Path(training_results.save_dir)
        save_model_info(experiment_path)
    else:
        print("❌ Training results not available to save model info.")


    print(f"\n🎉 Production deployment ready!")
    print(f"📁 Model files: qr_detection_max_accuracy/")
    print(f"🔧 Use 'qr_detector(image_path)' function for inference")

    # Test the inference function on a sample image
    if qr_detector is not None:
        test_images = list(TEST_IMAGES.glob('*.jpg'))[:1]
        if test_images:
            print(f"\n🧪 Testing inference function...")
            sample_img = test_images[0]
            detections = qr_detector(str(sample_img))
            print(f"   Sample detection on {sample_img.name}: {len(detections)} QR codes found")
            for i, det in enumerate(detections):
                print(f"     QR {i+1}: Confidence {det['confidence']:.3f}, Box: ({det['x1']:.0f}, {det['y1']:.0f}, {det['x2']:.0f}, {det['y2']:.0f})")

else:
    print("❌ No trained model available for export")

# Task
Debug and resolve the CUDA errors encountered during YOLOv8 training, then resume or restart training to achieve the desired accuracy, potentially increasing the number of epochs if necessary.

## Address cuda error

### Subtask:
Implement strategies to resolve the "CUDA error: resource already mapped", such as checking for conflicting processes, trying different CUDA versions if possible, or adding specific PyTorch environment variables.


**Reasoning**:
Implement strategies to resolve the CUDA error by checking for conflicting processes and setting the `PYTORCH_CUDA_ALLOC_CONF` environment variable before attempting to train the model again.



In [ ]:
# 🔧 Debugging CUDA Errors

print("🔍 Attempting to resolve CUDA errors...")

# Step 1: Check for conflicting processes (Manual Step - Instruct the user)
print("\nStep 1: Check for conflicting processes.")
print("Please manually check for any other applications or processes that might be using your GPU (e.g., other training sessions, games, video editing software).")
print("If you find any, close them and try re-running the training cell.")
print("On Windows, you can use Task Manager (Performance tab, GPU section).")
print("On Linux, you can use `nvidia-smi` in the terminal.")


# Step 2: Set PYTORCH_CUDA_ALLOC_CONF environment variable
print("\nStep 2: Setting PYTORCH_CUDA_ALLOC_CONF environment variable.")
try:
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
    print("✅ PYTORCH_CUDA_ALLOC_CONF set to 'max_split_size_mb:512'")
except Exception as e:
    print(f"❌ Failed to set PYTORCH_CUDA_ALLOC_CONF: {e}")

# Step 3 & 4: CUDA version switching, Driver/Hardware issues (Manual Steps - Instruct the user)
print("\nStep 3 & 4: CUDA Version / Driver / Hardware Check.")
print("If the error persists after closing conflicting processes and setting PYTORCH_CUDA_ALLOC_CONF, consider:")
print("  - Ensuring your NVIDIA drivers are up to date and compatible with your PyTorch and CUDA versions.")
print("  - If you have multiple CUDA versions installed, try switching to a different one compatible with your PyTorch installation.")
print("These steps are manual and depend on your system configuration.")

print("\nAttempting to re-run the training cell after applying potential fixes...")

# Re-run the training function
# Note: The training function `train_maximum_accuracy_model` is defined in cell xmKmqLzfly4Q.
# We call it again here to attempt training with the new environment variable setting.
print("\n🚀 Re-running maximum accuracy QR detection training...")
trained_model, training_results = train_maximum_accuracy_model()

if trained_model is not None:
    print("\n🎉 Training pipeline completed successfully after attempting fixes!")
    print("📁 Check 'qr_detection_max_accuracy' folder for results")
else:
    print("\n❌ Training failed again. Review the output above for new error messages.")
